In [7]:
import cv2


def facebox(faceNet,frame):
    frameheight = frame.shape[0]
    framewidth = frame.shape[1]
    blob = cv2.dnn.blobFromImage(frame,1.0,(227,277),[104,117,123],swapRB = False)
    faceNet.setInput(blob)
    detection = faceNet.forward()
    bboxs = []
    for i in range(detection.shape[2]):
        confidence = detection[0,0,i,2]
        if confidence > 0.7:
            x1 = int(detection[0,0,i,3]*framewidth)
            y1 = int(detection[0,0,i,4]*frameheight)
            x2 = int(detection[0,0,i,5]*framewidth)
            y2 = int(detection[0,0,i,6]*frameheight)
            bboxs.append([x1,y1,x2,y2])
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
                
        
    return frame, bboxs

In [8]:
faceProto = "C:\\Users\\tarun\\Desktop\\face and age\\opencv_face_detector.pbtxt"
faceModel = "C:\\Users\\tarun\\Desktop\\face and age\\opencv_face_detector_uint8.pb"

ageProto = "C:\\Users\\tarun\\Desktop\\face and age\\age_deploy.prototxt"
ageModel = "C:\\Users\\tarun\\Desktop\\face and age\\age_net.caffemodel"

genderProto = "C:\\Users\\tarun\\Desktop\\face and age\\gender_deploy.prototxt"
genderModel = "C:\\Users\\tarun\\Desktop\\face and age\\gender_net.caffemodel"

faceNet = cv2.dnn.readNet(faceModel,faceProto)
ageNet = cv2.dnn.readNet(ageModel,ageProto)
genderNet = cv2.dnn.readNet(genderModel,genderProto)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# time 23:30

In [9]:
video = cv2.VideoCapture(0)

padding = 20
while True:
    ret , frame = video.read()
    frame , bboxs = facebox(faceNet,frame)
    for bbox in bboxs:
        #face = frame[bbox[1]:bbox[3],bbox[0]:bbox[2]]
        
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]
        blob = cv2.dnn.blobFromImage(face,1.0,(227,227),MODEL_MEAN_VALUES,swapRB = False)
        genderNet.setInput(blob)
        genderPred = genderNet.forward()
        gender = genderList[genderPred[0].argmax()]
        
        ageNet.setInput(blob)
        agepred = ageNet.forward()
        age = ageList[agepred[0].argmax()]
        
        label = "{},{}".format(gender,age)
        #cv2.rectangle(frame,(bbox[1]-30),(bbox[2],bbox[1]),(0,255,0),-1)
        cv2.putText(frame,label,(bbox[0],bbox[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow("age-gender",frame)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
video.release()
cv2.destroyAllWindows()